In [ ]:
from semantic_communication.diffusion_super_res import DiffusionSuperRes, DiffusionUpscaler
import torch
import os
from PIL import Image

images_folder_input = "logs/upsampling/log-vit/snr_30/output"
images_folder_output = "logs/upsampling/log-vit/snr_30/diffusion"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

diffusion_super_res = DiffusionSuperRes(device)
# diffusion_upscaler = DiffusionUpscaler(device)

for filename in os.listdir(images_folder_input):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(images_folder_input, filename)

        # Open the image using PIL
        img_pil = Image.open(img_path).convert('RGB')

        # Run the diffusion model (using 200 as the inference step count)
        output_img = diffusion_super_res.inference(img_pil, 200)

        # Save the output image in the output folder with the same filename
        output_path = os.path.join(images_folder_output, filename)
        output_img.save(output_path)

        print(f"Processed and saved: {output_path}")

In [1]:
import os
from PIL import Image
from semantic_communication.metrics import calculate_lpips_similarity, calculate_psnr_np, calculate_psnr
import numpy as np
import torchvision.transforms as transforms

# Loop over SNR values from 1 to 30
snr_values = range(1, 31)
results = []

transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to 112x112
    transforms.ToTensor(),          # Convert the image to a tensor
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

for snr in snr_values:
    # Update the folder paths for the current SNR
    input_folder = f"logs/upsampling/log-vit/snr_{snr}/input"
    output_folder = f"logs/upsampling/log-vit/snr_{snr}/diffusion"
    
    # Ensure both folders exist
    if not os.path.exists(input_folder) or not os.path.exists(output_folder):
        print(f"Skipping SNR {snr} because the input or output folder does not exist.")
        continue

    # Initialize list to store LPIPS values
    lpips_values = []

    # Iterate through all images in the input folder
    for filename in os.listdir(input_folder):
        # Construct the input and output file paths
        index = filename.split('_')[1]  # Get the index from input_{index}.png
        input_image_path = os.path.join(input_folder, f"input_{index}")
        output_image_path = os.path.join(output_folder, f"output_{index}")

        input_img = Image.open(input_image_path).convert('RGB')
        output_img = Image.open(output_image_path).convert('RGB')
        
        # Convert to tensor
        input_img_tensor = transform(input_img).unsqueeze(0)  # Add batch dimension
        output_img_tensor = transform(output_img).unsqueeze(0)

        # Calculate LPIPS
        lpips_value = calculate_psnr(input_img_tensor, output_img_tensor)

        # Append the value to the list
        lpips_values.append(lpips_value)

    # Calculate the average LPIPS for this SNR
    if lpips_values:
        avg_lpips = np.mean(lpips_values)
        results.append({'snr': snr, 'avg_lpips': avg_lpips})
        print(f"SNR {snr}: Average LPIPS = {avg_lpips}")
    else:
        print(f"No valid comparisons found for SNR {snr}.")

# Output the results
for result in results:
    print(f"SNR {result['snr']}: Average LPIPS = {result['avg_lpips']:.4f}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
SNR 1: Average LPIPS = 18.840259552001953
SNR 2: Average LPIPS = 20.96728515625
SNR 3: Average LPIPS = 21.931808471679688
SNR 4: Average LPIPS = 22.478090286254883
SNR 5: Average LPIPS = 22.8260498046875
SNR 6: Average LPIPS = 23.048362731933594
SNR 7: Average LPIPS = 23.196651458740234
SNR 8: Average LPIPS = 23.307106018066406
SNR 9: Average LPIPS = 23.394601821899414
SNR 10: Average LPIPS = 23.441608428955078
SNR 11: Average LPIPS = 23.509063720703125
SNR 12: Average LPIPS = 23.58000373840332
SNR 13: Average LPIPS = 23.583158493041992
SNR 14: Average LPIPS = 23.608240127563477
SNR 15: Average LPIPS = 23.64699363708496
SNR 16: Average LPIPS = 23.651119232177734
SNR 17: Average LPIPS = 23.66393280029297
SNR 18: Average LPIPS = 23.687097549438477
SNR 19: Average LPIPS = 23.68998908996582
SNR 20: Average LPIPS = 23.744665145874023
SNR 21: Average LPIPS = 23.724538803100586
SNR 22